In [1]:
import itertools
import math
import random
import numpy

# Step 1
Simulate Rankings of Relevance for E and P


In [33]:
#Generating all possible combinations of relevance,
#where  0 - N, 1 - R, 2 - HR
possible_rankings = itertools.product([0,1,2], repeat=5)
P = [list(i) for i in possible_rankings]
E = P[:]

#Constructing all possible pairs
L = []
for item1 in P:
    for item2 in E:
        pair = []
        pair.append(item1)
        pair.append(item2)
        L.append(pair)
print "Number of possible rankings: ", len(L)

Number of possible rankings:  59049


# Step 2
Implement Evaluation Measures 

In [34]:
def precision(res, k):
    counter = 0
    for i in range(k):
        if res[i] != 0:
            counter += 1
    return 1.0*counter / k

def DCG(res, k):
    sum = 0
    for r in range(1,k+1):
        sum += (2**res[r-1] - 1)/math.log(1+r,2)
    return sum

def RBP(res, theta = 0.8):
    sum = 0
    for k in range(1, len(res)):
        sum += res[k-1] * theta**(k-1)*(1 - theta)
    return sum

N=1165
print "Example:"
print " Rankings: ", L[ N]
print " Precision: ", precision(L[N][0],k = 3),',', precision(L[N][1],k = 3)
print " DCG: ", DCG(L[N][0],k = 3),',',DCG(L[N][1],k = 3)
print " RBP: ", RBP(L[N][0]), ',', RBP(L[N][1])

Example:
 Rankings:  [[0, 0, 0, 1, 1], [2, 1, 0, 1, 1]]
 Precision:  0.0 , 0.666666666667
 DCG:  0.0 , 3.63092975357
 RBP:  0.1024 , 0.6624


# Step 3
 Calculate the 𝛥measure

In [35]:
def measure(pairs, measure, k=5):
    result = []
    for pair in pairs:
        if measure == "precision":
            P = precision(pair[0], k)
            E = precision(pair[1], k)
        
        if measure == "DCG":
            P = DCG(pair[0], k)
            E = DCG(pair[1], k)
            
        if measure == "RBP":
            P = RBP(pair[0])
            E = RBP(pair[1])
         
        diff = E - P
        
        if diff > 0:
            result.append([pair, diff])
            
    return result

res = measure(L, "DCG")

N=5432
print "Example: "
print " Rankings: ", res[N][0]
print " Measure difference: ", res[N][1]

Example: 
 Rankings:  [[0, 0, 2, 2, 1], [2, 0, 2, 2, 2]]
 Measure difference:  3.77370561447


# Step 4

In [36]:
#Team Draft Interleaving

#generate a list of random clicks
def get_clicks(l, number):
    ind_order = range(len(l))
    random.shuffle(ind_order)
    return ind_order[:number]

#a) create the interleaving list
def team_draft_interleave(pair):
    lists = [pair[0],pair[1]]
    l = []
    a = []
    while lists[0] or lists[1]:
        if lists[0] and lists[1]:
            first = random.randint(0,1)
        elif lists[0]:
            first = 0
        else:
            first = 1
        second = int(math.fabs(first-1))

        #pick from the first list
        doc = lists[first][0]
        l.append(doc)
        a.append(first)
        #delete the doc from the first list (top element) and the second list
        del lists[first][0]   
        try:
            lists[second].remove(doc)     
        except:
            pass
            
        #pick from the second list
        if lists[second]:
            doc = lists[second][0]
            l.append(doc)
            a.append(second) 
            #delete the doc from the second list (top element) and the first list
            del lists[second][0]        
            try:
                lists[first].remove(doc)    
            except:
                pass

    test = []
    for i in range(len(l)):
        test.append([l[i],a[i]])
    return test #l,a
    
#b) evaluate the interleaved list
def team_draft_evaluate(interleaved, clicks):
    counters = [0, 0]
    for index in clicks:
        counters[interleaved[index][1]]+=1
    return numpy.sign(counters[0]-counters[1])

def team_draft(pair, n_clicks=2):
    interleaved = team_draft_interleave(pair)
    clicks = get_clicks(interleaved, n_clicks)
    evaluation = team_draft_evaluate(interleaved, clicks)
    return evaluation

N=1232
print "TeamDraft Interleaving"
print "Example:"
print " Rankings: ", L[N]
res = team_draft(L[N])
print " Evaluation: ", res

TeamDraft Interleaving
Example:
 Rankings:  [[0, 0, 0, 1, 2], [0, 0, 1, 2, 2]]
 Evaluation:  0


In our implementation of the generic Probabilistic Interleaving, we also consider overlapping documents between the two rankings. To find which documents are the same, we are comparing the labels of the documents.

To make use of this generic algorithm with our rankings with relevance as labels, we have to alter the labels so every one of them is unique. In this way, we make sure that there are no overlapping documents between the two rankings, as is required by Note 5 a)

In [29]:
#Probabilistic Interleaving

#make all labels of the selected pair of rankings unique
def preprocess(pair):
    c = 0
    new_pair = ([], [])
    for i in range(2):
        for item in pair[i]:
            new_pair[i].append(str(item)+ str(c))
            c+=1
    return new_pair


#a) create the interleaving list
def prob_interleave(pair, tau = 3):
    #add to each entry of a ranking the id of the ranking it is from
    lists = [zip(pair[0],[0]*len(pair[0])), zip(pair[1],[1]*len(pair[1]))]
    result = []
    
    #construct the interleaving by iteratively
    #adding a pair of entries to the resulting list
    while lists[0] or lists[1]:
        #select the id of the ranking to draw an entry from
        if lists[0] and lists[1]:
            selected = random.randint(0,1)
        elif lists[0]:
            selected = 0
        else:
            selected = 1
        
        #calculate the softmax probabilities
        sum = 0
        for i in range(0, len(lists[selected])):
            sum += 1.0/((i+1)**tau)
            
        p = [(1.0/((i+1)**tau))/sum for i in range(0, len(lists[selected]))]
        
        #select a new entry with the softmax
        doc_index = numpy.random.choice(range(len(lists[selected])),1,p=p)
        doc = lists[selected][doc_index]
        result.append(doc)
        
        #delete the selected entry from both original rankings
        del lists[selected][doc_index]
        other_ind = int(math.fabs(selected-1))
        if lists[other_ind]:
            for i in range(len(lists[other_ind])):
                if lists[other_ind][i][0] == doc[0]:
                    del lists[other_ind][i]
                    break
    return result
        
    
#b) evaluate the interleaved list
def prob_evaluate(interleaved, clicks):
    return team_draft_evaluate(interleaved, clicks)

def prob(pair, n_clicks=2):
    interleaved = prob_interleave(pair)
    clicks = get_clicks(interleaved, n_clicks)
    return prob_evaluate(interleaved, clicks)
    
N=1234
print "Probabilistic Interleaving"
print "Example:"
print " Rankings", L[N]
pair = preprocess(L[N])
res = prob(pair)
print " Evaluation"

Probabilistic Interleaving
Example:
 Rankings [[0, 0, 0, 1, 2], [0, 0, 2, 0, 1]]
 Evaluation


D:\Work\Anaconda\envs\Python 2\lib\site-packages\ipykernel\__main__.py:40: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
D:\Work\Anaconda\envs\Python 2\lib\site-packages\ipykernel\__main__.py:44: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future


# Step 5

In [7]:

table = []
with open('YandexRelPredChallenge.txt', 'r') as f:
    for line in f:
        line = line.split()
        table.append(line)
f.closed

sum = 0
S = []
for line in table:
    if(line[2]) == 'Q':
        S.append([])
        S[-1].append(line[5:])
        S[-1].append([])
    if(line[2]) == 'C':
        c = line[-1] 
        for i in range(-1,-len(S) - 1, -1): #attribute the click to the last query that had this document as a result
#             if i!= -1:
#                 print line[0]
            if c in S[i][0]:
                S[i][1].append(c)
                break;

# create the sessions list (old way, session = yandex session)
# for key, group in itertools.groupby(table,lambda x: x[0]):
#     Q = []
#     for item in group:
#         if(item[2]) == 'Q':
#             Q.append([]) #the new query
#             Q[-1].append(item[5:]) #append the list of ranked documents
#             Q[-1].append([]) #append the clicks for this query (empty at first)
#         if(item[2]) == 'C':
#             c = item[-1] 
#             Q[-1][-1].append(c) #add the click on the last query    
#     S.append(Q)
# calculate the parameter for RCM (old way, session = yandex session)
# sum1 = 0
# sum2 = 0
# for s in S:
#     clicks = set()
#     for q in s:
#         for c in q[1]:
#             clicks.add(c)
#     sum1 += len(clicks)
    
#     docs = []
#     for q in s:
#         docs.extend(q[0])
#     docs = set(docs)
#     sum2 += len(docs)

# r = 1.0*sum1/sum2
# print 1.0*sum1/sum2


for i in range(5):
    print S[i]

[['7', '103', '51', '92', '43', '12', '73', '69', '27', '105'], []]
[['1625', '1627', '1623', '1626', '1624', '1622', '1619', '1621', '1620', '1618'], []]
[['2094', '2091', '2087', '2089', '2093', '2088', '2090', '2092', '2095', '2086'], []]
[['1625', '1627', '1623', '1626', '1624', '1622', '1619', '1621', '1620', '1618'], []]
[['17562', '1627', '1626', '1623', '2091', '17559', '17563', '17558', '17561', '17560'], ['17562', '1627', '1626']]


In [8]:
def RCM_parameter(S):
    sum1 = 0
    sum2 = 0
    for s in S:
        clicks = set()
        for c in s[1]:
            clicks.add(c)
        sum1 += len(clicks)

        sum2 += len(s[0])

    return 1.0*sum1/sum2

def RCM(r,l):
    c = []
    for i in range(len(l)):
        p = random.uniform(0, 1)
        if p < r:
            c.append(i)
    return c

r = RCM_parameter(S)
for i in range(10):
    print RCM(r,[1,2,3,4,5,6,7,8,9,0])


[0]
[4, 7]
[6]
[0]
[1]
[]
[5, 8]
[2]
[4]
[1, 5]


In [9]:
def get_Sr(queries, r):
    return  filter(lambda query: query[0][r-1] in query[1] , queries)

def get_lambda_r(queries, r):
    Sr = get_Sr(queries, r)
    n = len(Sr)
    sum = 0
    for s in Sr:
        indices = [-1] + sorted([s[0].index(doc1) for doc1 in s[1]])
        last_clicked = indices[-1]+1
        additive = 0
        if last_clicked != r:
            additive = 1
        sum += additive
    return sum/float(len(Sr))
        
def get_lambdas(queries, maxr=10):
    return [get_lambda_r(queries, r) for r in range(1, maxr+1)]

#get the clicks for the ranked document list, where
#'l_r' is the probability of continuing examination when the document at rank 'r' was clicked
#'a_u' is the attractiveness of document 'u' for this specific query
def SDCM(a_u,l_r):
    clicks = []
    for i in range(len(a_u)):
        p = random.uniform(0,1) #choose if to click
        if p < a_u[i]:
            clicks.append(i) 
            p = random.uniform(0,1) #choose if to stop examining
            if p < l_r[i]:
                continue
            else:
                break
    return clicks


lambdas = get_lambdas(S)
for i in range(10):
    print SDCM([0.8, 0.9, 0.3, 0.5, 0.6, 0.4], lambdas)#[0.5, 0.6, 0.4, 0.3, 0.2, 0.3])
            
        

[2]
[0]
[1, 2]
[0]
[1, 3, 4]
[0]
[0, 1, 3, 4]
[0, 1]
[1, 4, 5]
[0, 1, 3, 4, 5]


# Step 6